In [152]:
import pandas as pd
import seaborn as sns
import json
import os
import numpy as np

def set_custom_seaborn_style():
    custom_params = {
        "axes.facecolor": "white",
        "grid.color": "#e6e6e6",
        "grid.linestyle": "--",
        "axes.edgecolor": "#cccccc",
        "axes.labelsize": 14,
        "axes.titlesize": 16,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        "xtick.color": "#555555",
        "ytick.color": "#555555",
        "legend.frameon": False,
        "legend.fontsize": 12,
        "font.family": "sans-serif",
        "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],
    }
    sns.set_theme(style="whitegrid", rc=custom_params)

set_custom_seaborn_style()

## All Business and Finance Data

In [3]:
df = pd.read_csv('../../data/task_lists/business_and_financial_operations_occupations.csv')

In [4]:
df.shape[0]

1008

In [5]:
df_core = df[df['task_type']=='Core']
df_core.shape[0]

675

In [6]:
# num of occupations
print('overall occupations', len(df['title'].unique()))
print('core', len(df_core['title'].unique()))

overall occupations 48
core 43


In [186]:
# read in tools materials list to create overview table

mt = pd.read_csv('/Users/htr365/Documents/PhD/21_automatisation/gpt_eval/data/exam_approach/material_lists/claude-3-7-sonnet-20250219/task_list_business_and_financial_operations_occupations_CORE.csv')

tool_columns = [col for col in mt.columns if col.startswith('tool')]
material_columns = [col for col in mt.columns if col.startswith('material')]
relevant_columns = [item for item in material_columns if 'other' not in item.lower()] + [item for item in tool_columns if 'other' not in item.lower()] 

mt_binary = (mt[relevant_columns] =='Required').astype(int)

multiindex_tuples = [('Tools', col.replace("tools.", "").strip()) if 'tool' in col else ('Materials', col.replace("materials.", "").strip()) for col in relevant_columns]
overview_per_occupation = pd.concat([mt['occupation'], mt_binary],axis=1).groupby('occupation').sum()
overview_per_occupation.columns = pd.MultiIndex.from_tuples(multiindex_tuples, names=['Materials','Tools'])

overview_per_occupation['Core tasks per occupation'] = mt.groupby('occupation').size()
overview_per_occupation['Remote possible'] = mt[mt['can_be_performed_remotely']==True].groupby('occupation').size()
overview_per_occupation['Pratical assessment feasible'] = mt[mt['feasiblity_practical']==True].groupby('occupation').size()
#overview_per_occupation['\% labeled as automatable\n by at least one rater'] = np.round(automation_potential,0)


overview_per_occupation = overview_per_occupation[['Core tasks per occupation','Remote possible', 'Pratical assessment feasible','Materials','Tools']]



In [187]:
exclusion_list = pd.read_csv('../../data/exam_approach/exclusion_lists/only_data_text_CORE.csv',index_col=0).rename(columns={'0':'task_id'})
overview_per_occupation['Analysed tasks - absolute number'] = mt[~mt['task_id'].isin(exclusion_list['task_id'])].groupby('occupation').size().astype(int)
overview_per_occupation.loc[ overview_per_occupation['Analysed tasks - absolute number'].isna(),'Analysed tasks - absolute number']=0
overview_per_occupation['Analysed tasks [\% of all tasks]'] = np.round(overview_per_occupation['Analysed tasks - absolute number'] / overview_per_occupation['Core tasks per occupation'] * 100)

In [188]:
overview_per_occupation 

Materials                                          Core tasks per occupation  \
Tools                                                                          
occupation                                                                     
Accountants and Auditors                                                  20   
Agents and Business Managers of Artists, Perfor...                         9   
Appraisers and Assessors of Real Estate                                   26   
Budget Analysts                                                           12   
Business Continuity Planners                                              20   
Buyers and Purchasing Agents, Farm Products                                6   
Claims Adjusters, Examiners, and Investigators                            21   
Compensation, Benefits, and Job Analysis Specia...                        14   
Compliance Officers                                                        6   
Coroners                                                                  17   
Cost Estimators                                                           12   
Credit Analysts                                                            7   
Credit Counselors                                                         21   
Customs Brokers                                                           21   
Environmental Compliance Inspectors                                       15   
Equal Opportunity Representatives and Officers                            11   
Farm Labor Contractors                                                     6   
Financial Examiners                                                       16   
Financial Quantitative Analysts                                           16   
Fraud Examiners, Investigators and Analysts                               21   
Fundraisers                                                               24   
Government Property Inspectors and Investigators                           3   
Human Resources Specialists                                               19   
Insurance Appraisers, Auto Damage                                          7   
Insurance Underwriters                                                     7   
Labor Relations Specialists                                               25   
Loan Officers                                                             14   
Logisticians                                                              22   
Logistics Analysts                                                        22   
Logistics Engineers                                                       27   
Management Analysts                                                       10   
Market Research Analysts and Marketing Specialists                        13   
Meeting, Convention, and Event Planners                                   16   
Online Merchants                                                          24   
Personal Financial Advisors                                               17   
Purchasing Agents, Except Wholesale, Retail, an...                        19   
Regulatory Affairs Specialists                                            24   
Search Marketing Strategists                                              25   
Sustainability Specialists                                                14   
Tax Examiners and Collectors, and Revenue Agents                           6   
Tax Preparers                                                             12   
Training and Development Specialists                                      18   
Wholesale and Retail Buyers, Except Farm Products                         10   

Materials                                          Remote possible  \
Tools                                                                
occupation                                                           
Accountants and Auditors                                        19   
Agents and Business Managers of Artists

In [189]:
# # Function to handle multirow headers correctly, adding an empty row if the title is short
# def split_headers_multirow(header, max_words=5):
#     words = header.split()
#     if len(words) > max_words:
#         # If the header is long, split it into two lines
#         mid = len(words) // 2  # Split in the middle
#         # Return two-row multirow structure
#         return f"\\multirow{{2}}{{*}}{{{' '.join(words[:mid])}}} { {' '.join(words[mid:])} }"
#     else:
#         # If the header is short, just use multirow without splitting
#         return f"\\multirow{{2}}{{*}}{{{header}}} {} "

def convert_to_latex(df, caption, table_name):
    # split column names if necessary
    # df.columns = [split_headers_multirow(col) for col in df.columns]
    # round numbers
    df =df.apply(lambda col: pd.to_numeric(col, errors='ignore').astype(int, errors='ignore'))
    df[df.select_dtypes(include=['float64']).columns] = df.select_dtypes(include=['float64']).applymap(lambda x: f"{x:.2f}")
    # convert to latex
    latex_table = df.to_latex(index=False, escape=False)
    table_label = '{tab:'+table_name+'}'
    caption = '{'+caption+'}'
    print(latex_table)
    table_environment = f"""
    \\begin{{table}}[h]
    \\resizebox*{{\\textwidth}}{{!}}{{%
        \\centering
        {latex_table}
        }}
        \\caption{caption}
            \\label{table_label}
    \\end{{table}} """
    print(table_environment)
    with open("../../results/tables/"+table_name+".tex", "w") as f:
        f.write(table_environment)


In [190]:

convert_to_latex(overview_per_occupation.reset_index(),\
     "Overview of number of tasks per occupation and share of tasks labeled as having the potential to be fully automated by conversational AI agents.",\
         'occupation_overview')

\begin{tabular}{lrrrrrrrrrrrrrrrrrr}
\toprule
occupation & Core tasks per occupation & Remote possible & Pratical assessment feasible & \multicolumn{6}{r}{Materials} & \multicolumn{7}{r}{Tools} & Analysed tasks - absolute number & Analysed tasks [\% of all tasks] \\
 &  &  &  & Text & Data & Images & Audio files & Video files & Virtual labs or sandbox environments & Coding & Spreadsheets & Text editor & PDF viewer & Presentation software & Online search engine & Image Generator &  &  \\
\midrule
Accountants and Auditors & 20 & 19 & 19 & 19 & 19 & 4 & 0 & 1 & 16 & 0 & 19 & 17 & 19 & 3 & 5 & 0 & 1 & 5 \\
Agents and Business Managers of Artists, Performers, and Athletes & 9 & 9 & 9 & 9 & 9 & 4 & 2 & 2 & 3 & 0 & 5 & 9 & 9 & 3 & 7 & 0 & 0 & 0 \\
Appraisers and Assessors of Real Estate & 26 & 24 & 24 & 24 & 24 & 23 & 0 & 2 & 8 & 0 & 17 & 18 & 23 & 3 & 16 & 0 & 0 & 0 \\
Budget Analysts & 12 & 12 & 12 & 12 & 12 & 0 & 0 & 0 & 0 & 0 & 12 & 11 & 12 & 3 & 1 & 0 & 9 & 75 \\
Business Continuity Plan

/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_8699/3556868670.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df =df.apply(lambda col: pd.to_numeric(col, errors='ignore').astype(int, errors='ignore'))
/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_8699/3556868670.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df =df.apply(lambda col: pd.to_numeric(col, errors='ignore').astype(int, errors='ignore'))
/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_8699/3556868670.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df =df.apply(lambda col: pd.to_numeric(col, errors='ignore').astype(int, errors='ig

# ARCHIVE

## Labeling

In [7]:
labels_je = pd.read_csv('/Users/htr365/Documents/PhD/21_automatisation/gpt_eval/data/manual_automation_labels/business_and_financial_operations_occupations_JE.csv')
labels_mr = pd.read_csv('/Users/htr365/Documents/PhD/21_automatisation/gpt_eval/data/manual_automation_labels/business_and_financial_operations_occupations_MR.csv')
labels_mr.rename(columns={'Maria label':'label_MR'}, inplace=True)
df = df.merge(labels_mr[['task_id','label_MR']], how='left',on='task_id').merge(labels_je[['task_id','label_JE']]).dropna(subset=['label_MR','label_JE'])

In [8]:
# create columns to indicate at least one or both automation potential labels
df['at_least_one'] = (df['label_JE'] + df['label_MR'])>0
df['both'] = (df['label_JE'] * df['label_MR'])>0

In [9]:
df[(df['at_least_one']==1)].shape[0]

639

In [14]:

# core tasks labelled as automatable by at least one person
df[(df['task_type']=='Core') * (df['at_least_one']==1)].shape[0]

435

In [173]:
df.to_csv('/Users/htr365/Documents/PhD/21_automatisation/gpt_eval/data/task_lists/E1E2_OR_E1E2/business_and_financial_operations_occupations_labels.csv')

In [28]:
#occupation_counts =df.groupby('title').size()
df = df[df['task_type']=='Core']
occupation_core_counts = df[df['task_type']=='Core'].groupby('title').size()
automation_potential = df.groupby('title')[['at_least_one']].mean()*100

In [29]:
occupation_overview = occupation_core_counts.reset_index().rename(columns={0:'Number of core tasks'})\
.merge(automation_potential.reset_index(), on='title')

In [32]:
occupation_overview.rename(columns={'title':'Occupation', 0:'Number of tasks','at_least_one':"""\% labeled as automatable
                                    by at least one rater"""}, inplace=True)


In [94]:
mt[(mt['materials.Audio files']=='Required')| (mt['materials.Video files']=='Required')|(mt['materials.Images']=='Required') |(mt['materials.Virtual labs or sandbox environments']=='Required')].shape
  #  |(mt['tools.Image Generator'] =='Required')|(mt['tools.Online search engine']==' Required') | (mt['tools.Presentation software']=='Required')\
#].shape[0]

(496, 31)

In [99]:
exclusion_tools = ['Presentation software', 'Image Generator', 'Online search engine']
exclusion_tools = ['tools.' + item for item in exclusion_tools]  # Modify tool names to match the column names
excluded_ids = list(mt.loc[(mt[exclusion_tools] == 'Required').sum(axis=1) >= 1, 'task_id'])
len(excluded_ids)

423

In [107]:
mt.loc[(mt[exclusion_tools] == 'Required').sum(axis=1) >= 1][exclusion_tools]

,tools.Presentation software,tools.Image Generator,tools.Online search engine
1,Required,Not Required,Required
2,Not Required,Not Required,Required
6,Not Required,Not Required,Required
7,Not Required,Not Required,Required
8,Not Required,Not Required,Required
...,...,...,...
667,Required,Not Required,Not Required
669,Not Required,Not Required,Required
670,Not Required,Not Required,Required
671,Required,Not Required,Not Required


In [126]:
excluded = mt[(mt['tools.Image Generator'] =='Required') |(mt['tools.Online search engine']=='Required') | (mt['tools.Presentation software']=='Required')\
    | (mt['materials.Audio files']=='Required')| (mt['materials.Video files']=='Required')|(mt['materials.Images']=='Required') |(mt['materials.Virtual labs or sandbox environments']=='Required')]

In [125]:
excluded

(587, 31)

In [119]:
mt[(mt['materials.Audio files']=='Not Required')*(mt['materials.Video files']=='Not Required')*(mt['materials.Images']=='Not Required') *(mt['materials.Virtual labs or sandbox environments']=='Not Required')\
    *(mt['tools.Image Generator'] =='Not Required')*(mt['tools.Online search engine']=='Not Required') * (mt['tools.Presentation software']=='Not Required')\
].shape[0]


66

In [131]:
(mt['feasiblity_practical']==).sum()

np.int64(0)

In [137]:
mt_ex = mt[~mt['task_id'].isin(excluded['task_id'])]

In [124]:
mt['task_id'].isin(excluded['task_id'])

TypeError: tuple indices must be integers or slices, not str

In [139]:
mt_ex[mt_ex['can_be_performed_remotely'] == False].shape[0]

22

In [86]:
mt[mt.isna().sum(axis=1)>=1]

,Unnamed: 0.1,Unnamed: 0,row,prompt,task_id,occupation,task_description,can_be_performed_remotely,feasiblity_practical,chain_of_thought,...,materials.Images,materials.Audio files,materials.Video files,materials.Virtual labs or sandbox environments,materials.Other.name,materials.Other.classification,required_tools,required_tools_standard,required_materials,required_materials_standard
1,1,1,1,\nTask ID: 21163.0\n\nYour assignment is to de...,21163.0,"Agents and Business Managers of Artists, Perfo...",Send samples of clients' work and other promot...,True,True,This task can be practically tested remotely b...,...,Required,Required,Required,Not Required,NaN,NaN,"['Text editor', 'PDF viewer', 'Presentation so...","['Text editor', 'PDF viewer', 'Presentation so...","['Text', 'Data', 'Images', 'Audio files', 'Vid...","['Text', 'Data', 'Images', 'Audio files', 'Vid..."
5,5,5,5,\nTask ID: 20713.0\n\nYour assignment is to de...,20713.0,"Wholesale and Retail Buyers, Except Farm Products","Recommend mark-up rates, mark-down rates, or m...",True,True,This task can be practically tested remotely b...,...,Required,Not Required,Not Required,Not Required,NaN,NaN,"['Spreadsheets', 'Text editor', 'PDF viewer']","['Spreadsheets', 'Text editor', 'PDF viewer']","['Text', 'Data', 'Images']","['Text', 'Data', 'Images']"
10,10,10,10,\nTask ID: 1145.0\n\nYour assignment is to det...,1145.0,"Purchasing Agents, Except Wholesale, Retail, a...","Analyze price proposals, financial reports, an...",True,True,This task can be effectively tested remotely b...,...,Required,Not Required,Not Required,Not Required,NaN,NaN,"['Spreadsheets', 'Text editor', 'PDF viewer', ...","['Spreadsheets', 'Text editor', 'PDF viewer', ...","['Text', 'Data', 'Images']","['Text', 'Data', 'Images']"
11,11,11,11,\nTask ID: 1159.0\n\nYour assignment is to det...,1159.0,"Purchasing Agents, Except Wholesale, Retail, a...",Formulate policies and procedures for bid prop...,True,True,Formulating procurement policies and procedure...,...,Not Required,Not Required,Not Required,Not Required,Sample procurement scenarios,Required,"['Spreadsheets', 'Text editor', 'PDF viewer', ...","['Spreadsheets', 'Text editor', 'PDF viewer', ...","['Text', 'Data', 'Sample procurement scenarios']","['Text', 'Data']"
14,14,14,14,\nTask ID: 1153.0\n\nYour assignment is to det...,1153.0,"Purchasing Agents, Except Wholesale, Retail, a...",Study sales records and inventory levels of cu...,True,True,This task can be practically tested remotely b...,...,Not Required,Not Required,Not Required,Not Required,NaN,NaN,"['Spreadsheets', 'Text editor', 'PDF viewer', ...","['Spreadsheets', 'Text editor', 'PDF viewer', ...","['Text', 'Data']","['Text', 'Data']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,188,188,502,\nTask ID: 21551.0\n\nYour assignment is to de...,21551.0,Appraisers and Assessors of Real Estate,Inspect new construction and major improvement...,False,False,Physical property inspection requires direct o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629,194,194,520,\nTask ID: 21569.0\n\nYour assignment is to de...,21569.0,Appraisers and Assessors of Real Estate,Photograph interiors and exteriors of properti...,False,False,This task requires physically visiting propert...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656,221,221,615,\nTask ID: 3417.0\n\nYour assignment is to det...,3417.0,Loan Officers,Handle customer complaints and take appropriat...,True,True,A practical remote exam for handling customer ...,...,Not Required,Required,Not Required,Required,NaN,NaN,"['Text editor', 'PDF viewer', 'Customer Relati...","['Text editor', 'PDF viewer']","['Text', 'Data', 'Audio files', 'Virtual labs ...","['Text', 'Data', 'Audio files', 'Virtual labs ..."
663,228,228,637,\nTask ID: 21165.0\n\nYour assignment is to de...,21165.0,Tax Preparers,Schedule appointments with clients.,True,True,Scheduling appointments is a core administrati...,...,Not Required,Not Required,Not Requ